# Information Retrieval and Web Analytics

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
# from config import *

# GROUP NAMES:
- Judith Camacho 218863 
- Jordi Marín    207552 
- Xavier Vives   218900

# **Project 1**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
from ast import literal_eval
import time
import math
import re
import json
from datetime import datetime
import collections
from numpy import linalg as la

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data_path = "/content/drive/Shareddrives/IRWA/Project/Part 3/data/"
docs_path = data_path+'tweet_df.csv'

tweet_df = pd.read_csv(docs_path, index_col=[0])
tweet_df['processed_text'] = tweet_df['processed_text'].apply(lambda x: literal_eval(x))
tweet_df.head()

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,Doc,processed_text
1575918182698979328,So this will keep spinning over us until 7 pm…...,Suz👻,2022-09-30 18:39:08,['HurricaneIan'],0,0,https://twitter.com/Suz/status/157591818269897...,doc_1,"[keep, spin, us, 7, pmgo, away, alreadi]"
1575918151862304768,Our hearts go out to all those affected by . W...,Lytx,2022-09-30 18:39:01,['HurricaneIan'],0,0,https://twitter.com/Lytx/status/15759181518623...,doc_2,"[heart, go, affect, wish, everyon, road, curre..."
1575918140839673873,Kissimmee neighborhood off of Michigan Ave. \n,Christopher Heath,2022-09-30 18:38:58,['HurricaneIan'],0,0,https://twitter.com/Christopher_Heath/status/1...,doc_3,"[kissimme, neighborhood, michigan, ave]"
1575918135009738752,I have this one tree in my backyard that scare...,alex ✨,2022-09-30 18:38:57,"['scwx', 'HurricaneIan']",0,0,https://twitter.com/alex_/status/1575918135009...,doc_4,"[one, tree, backyard, scare, poltergeist, tree..."
1575918119251419136,"I pray for everyone affected by , but...",Tess 💋,2022-09-30 18:38:53,['HurricaneIan'],0,0,https://twitter.com/Tess_/status/1575918119251...,doc_5,"[pray, everyon, affect, associ, winknew, sympa..."


In [ ]:
tweet_df_doc = tweet_df.copy(deep=True)
tweet_df_doc = tweet_df_doc.set_index('Doc')
tweet_df_doc.head()

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,processed_text
Doc,,,,,,,,
doc_1,So this will keep spinning over us until 7 pm…...,Suz👻,2022-09-30 18:39:08,['HurricaneIan'],0,0,https://twitter.com/Suz/status/157591818269897...,"[keep, spin, us, 7, pmgo, away, alreadi]"
doc_2,Our hearts go out to all those affected by . W...,Lytx,2022-09-30 18:39:01,['HurricaneIan'],0,0,https://twitter.com/Lytx/status/15759181518623...,"[heart, go, affect, wish, everyon, road, curre..."
doc_3,Kissimmee neighborhood off of Michigan Ave. \n,Christopher Heath,2022-09-30 18:38:58,['HurricaneIan'],0,0,https://twitter.com/Christopher_Heath/status/1...,"[kissimme, neighborhood, michigan, ave]"
doc_4,I have this one tree in my backyard that scare...,alex ✨,2022-09-30 18:38:57,"['scwx', 'HurricaneIan']",0,0,https://twitter.com/alex_/status/1575918135009...,"[one, tree, backyard, scare, poltergeist, tree..."
doc_5,"I pray for everyone affected by , but...",Tess 💋,2022-09-30 18:38:53,['HurricaneIan'],0,0,https://twitter.com/Tess_/status/1575918119251...,"[pray, everyon, affect, associ, winknew, sympa..."


In [ ]:
def create_index(lines):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of tweets
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)

    for doc,terms in zip(lines['Doc'].values, lines['processed_text'].values):  # Remember, lines contain all documents from file
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                current_page_index[term][1].append(position)
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term] = [doc, array('I', [position])]  #'I' indicates unsigned int (int in Python)
        # merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
    return index

In [ ]:
start_time = time.time()
index = create_index(tweet_df)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

Total time to create the index: 0.47 seconds


In [ ]:
def create_tfidf(lines, num_documents):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of tweets
    num_documents -- total number of documents
    
    Returns:
    tf - normalized term frequency for each term in each document
    idf - inverse document frequency of each term
    """

    tf = defaultdict(list)  # term frequencies of terms in documents (documents in the same order as in the main index)
    df = defaultdict(int)  # document frequencies of terms in the corpus
    idf = defaultdict(float)

    for doc,terms in zip(lines['Doc'].values, lines['processed_text'].values):
        ## ===============================================================        
        ## create the index for the **current page** and store it in current_page_index
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms):  ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corresponding list
                current_page_index[term][1].append(position)
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[doc, array('I',[position])] #'I' indicates unsigned int (int in Python)

        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm = 0
        for term, posting in current_page_index.items():
            # posting will contain the list of positions for current term in current document. 
            # posting ==> [current_doc, [list of positions]] 
            # you can use it to infer the frequency of current term.
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)

        # calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4)) ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term] += 1 # increment DF for current term

        # Compute IDF following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(num_documents/df[term])), 4)

    return tf, idf

In [ ]:
start_time = time.time()
num_documents = len(tweet_df)
tf, idf = create_tfidf(tweet_df, num_documents)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the index: 187.77 seconds


In [ ]:
k = 50

sort_df = pd.Series(data=np.fromiter(idf.values(),dtype=float),index=idf.keys())
sort_df = sort_df.sort_values()#ascending=False)
#sort_df.head(k)

Now we build the queries list. Note that these queries are from the Part 2 deliverable. We initially had other queries, but based on the word relevance we computed, we finally chose these 5 queries.

In [ ]:
queries = ['power outage in carolina',
           'school closed',
           'deaths hurricane florida',
           'how to help hurricane',
           'president speech']

In [ ]:
def build_terms(tweet_text):
    '''
    input: the text of a tweet
    function: pre-process the text as said above
    output: a list of strings of the processed text
    '''
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))

    tweet_text = tweet_text.lower() ## Transform in lowercase
    tweet_text = re.sub(r'[^\w\s]', '', tweet_text) #remove punctuation marks and hashtags using regex

    tweet_text = tweet_text.split() ## Tokenize the text to get a list of terms
    tweet_text = [x for x in tweet_text if x not in stop_words]  ##eliminate the stopwords
    tweet_text = [stemmer.stem(x) for x in tweet_text] ## perform stemming 
    return tweet_text

# Part 1

Now we buil the function search that for a given query retrieves the documents that have a high score (can be idf-tf score, adhoc score--> our scoring model or bm25 score) for that query. 

In [ ]:
def search(query, index, model='idf-tf'):
    """
    Arguments: 
    - index -- inverted index data structure
    - query: query we are interested in
    - model: type of scoring we want. Can be idf-tf, adhoc or bm25
    Returns: 
    The list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs = [posting[0] for posting in index[term]]
            
            # docs = docs Union term_docs
            docs |= set(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    if model == 'adhoc':
      ranked_docs,_ = rank_documents_adhoc(query, docs, index, idf, tf, adhoc_score_df)
    elif model == 'idf-tf':
      ranked_docs,_ = rank_documents(query, docs, index, idf, tf)
    elif model == 'bm25':
      ranked_docs = rank_documents_bm25(query, bm25)
    else:
      print("Invalid model:", model)
      ranked_docs = []
    return ranked_docs

## a) Tf-Idf model + cosine similarity


This function is reused from the previous lab 

In [ ]:
def rank_documents(terms, docs, index, idf, tf, sorted=True):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    sorted -- if we want that the function returns list ordered or not. Yes, by default.
    
    Returns:
    Print the list of ranked documents and the doc_id and its tf-idf value.
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue
        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex] = query_terms_count[term] / query_norm * idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):
                     
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  

    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    
    doc_scores = [[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items()]
    if sorted:
      doc_scores.sort(reverse=True)
      result_docs = [x[1] for x in doc_scores]
      #print document titles instead if document id's
      #result_docs=[ title_index[x] for x in result_docs ]
      if len(result_docs) == 0:
          print("No results found")
      #print ('\n'.join(result_docs), '\n')
    else:
      result_docs = []
    return result_docs, doc_scores

In [ ]:
print("Queries:")
#query = input()
for query in queries:
  ranked_docs = search(query, index, model='idf-tf')
  top = 20
  print("\n======================\nQuery: {}".format(query))
  print("\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
  for d_id in ranked_docs[:top]:
      print("{}: {}".format(d_id, tweet_df_doc.loc[d_id]['Tweet']))

Queries:

Query: power outage in carolina

Top 20 results out of 552 for the searched query:

doc_755: We have 10,000  responders prepared for outages across the  including &gt;6,000 lineworkers. We will respond to outages as soon as it’s safe to do so. Customers should expect outages and have a plan.     
doc_2607: HOW TO REPORT AN OUTAGE WHERE YOU ARE: 
- North Carolina: 
- South Carolina: 
   
doc_337: Up to 40,000+ power outages in NC now. 

SC has 115,000. 

  
doc_1302: We are having power outages here. Stay safe everyone and check in on family and friends  
doc_686: Landfall of  is imminent just up the coast from Charleston as power outages begin to mount in coastal South Carolina.

Stay with  for the latest. 
doc_1622: With  🌀 passing through today, remember to report any power outages ⚡ directly to your service provider.
➡️ : 
➡️ : 
➡️  
doc_1850: The current electrical customers in Florida without power is 1.8M, after a peak of 2.6M.

Track power outages in real-time: .

    

## b) Our model, called adhoc model

First we extract the range of values of the number of likes and retweets.

In [ ]:
tweet_df_doc.Likes.unique()

array([   0,    1,    2,   17,    5,    3,    4,   15,   10,    6,   19,
         13,    8,   14,   74,   11,   29,  165,   25,    9,   12,   16,
         18,    7,   97,   38,  219,   35,   31,   60,   39,   20,   55,
         33,   32, 1326,   23,  779,   58,  176,  119,   34,   65,   47,
         30,   21,  129,  197,   22,   36,   42,   28,   48,   27,   41,
         24,  360,   80,   26,   67,   66,   53,   46,  173,   37,  184,
         43,  163,   61,   62,   64,  221,   84,   54,   75,   79,   40,
        124,  143,   87,  149,  112,   44,   49, 1120,  534,   73,   51,
        372,  151,  228,  315,   77,  178,  117,  100])

In [ ]:
tweet_df_doc.Retweets.unique()

array([  0,   2,   3,   1,  11,  14,  15,   4,   5,  26,   6,   7,  20,
        19,   8,  12,  10,  18,  28,  51,   9,  13,  21,  16, 318, 362,
        17,  50,  37,  36, 121,  25,  40,  38,  22,  54,  31,  30,  92,
        42,  44,  29,  47, 102,  46,  85,  23,  77])

Our score will be: the number of likes of the current doc normalized to the range of values of the dataset. The same with the number of retweets and then we sum them. This is what the next function computes:

In [ ]:
def score_by_likes_rt(nlikes, nretweets):
    min_likes = 0 
    min_rts = 0 
    max_likes = 1000 #based ont he values we have on our df
    max_rts = 300 #based ont he values we have on our df
    score_adhoc_likes = nlikes/(max_likes-min_likes) 
    score_adhoc_rts = nretweets/(max_rts-min_rts)

    return score_adhoc_likes + score_adhoc_rts

We create a new dataframe with just the columns Likes and Retweets and based on that we compute our adhoc score. We store it in a new column.

In [ ]:
adhoc_score_df = tweet_df_doc[["Likes", "Retweets"]]


In [ ]:
adhoc_score_df["adhoc_score"] = adhoc_score_df.apply(lambda x: score_by_likes_rt(x.Likes, x.Retweets), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
adhoc_score_df

,Likes,Retweets,adhoc_score
Doc,,,
doc_1,0,0,0.000000
doc_2,0,0,0.000000
doc_3,0,0,0.000000
doc_4,0,0,0.000000
doc_5,0,0,0.000000
...,...,...,...
doc_3996,2,0,0.002000
doc_3997,0,0,0.000000
doc_3998,16,8,0.042667


Next we build the ranking of documents with respect to a given query based on our new score.
We pick the tf-idf score and cosine similarity and we multiply it by our adhoc score buit from the likes and retweets. This way, we are giving some relevance to these 2 variables because they tak einto consideration the "popularity" of the document.

In [ ]:
def rank_documents_adhoc(terms, docs, index, idf, tf, adhoc, sorted=True):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    adhoc -- the adhoc value for a given document
    sorted -- if we want that the function returns list ordered or not. Yes, by default.
    
    Returns:
    Print the list of ranked documents and the doc_id and its tf-idf value.
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue
        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex] = query_terms_count[term] / query_norm * idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):
                     
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  

    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    
    doc_scores = [[np.dot(curDocVec, query_vector)*adhoc.adhoc_score[doc], doc] for doc, curDocVec in doc_vectors.items()]
    if sorted:
      doc_scores.sort(reverse=True)
      result_docs = [x[1] for x in doc_scores]
      #print document titles instead if document id's
      #result_docs=[ title_index[x] for x in result_docs ]
      if len(result_docs) == 0:
          print("No results found, try again")
          query = input()
          docs = search(query, index, model='adhoc')
      #print ('\n'.join(result_docs), '\n')
    else:
      result_docs = []
    return result_docs, doc_scores

In [ ]:
print("Queries:")
#query = input()
for query in queries:
  ranked_docs = search(query, index, model='adhoc')
  top = 20
  print("\n======================\nQuery: {}".format(query))
  print("\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
  for d_id in ranked_docs[:top]:
      print("{}: {}".format(d_id, tweet_df_doc.loc[d_id]['Tweet']))

Queries:

Query: power outage in carolina

Top 20 results out of 552 for the searched query:

doc_686: Landfall of  is imminent just up the coast from Charleston as power outages begin to mount in coastal South Carolina.

Stay with  for the latest. 
doc_581: Driving across the state from Miami, food truck Monster Burgers helped WCK serve 1,000 meals to families in Cape Coral yesterday.  damaged a majority of homes here &amp; there is no power or water. For many, this was the first hot meal since the storm.  
doc_3636: Hurricane Ian heads towards South Carolina after striking Florida.

The death toll in Florida has risen to 21. We are praying from Israel for all those affected. Our hearts are with you.  
doc_2684: 1140a: Eye of  just 40-50 miles offshore South Carolina. Landfall between Charleston and Myrtle Beach next few hours. 
doc_1335: WATCH: After ravaging Florida, a strengthened Hurricane Ian has made landfall in South Carolina and is predicted to enter North Carolina soon.      

## c) BM25

Now we implement the bm25 model.

In [ ]:
# !pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rank_bm25 import *

We tokenize the corpus, we load the bm25 model using the library.

In [ ]:
tokenized_corpus = tweet_df_doc['processed_text'].values
bm25 = BM25Okapi(tokenized_corpus)


Next we build the rank fucntion that retrieves the most relevant documents to a query based on the BM25 model.

In [ ]:
def rank_documents_bm25(terms, bm25):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    b25 -- bm25 model loaded from the library
    
    Returns:
    Print the list of ranked documents and the doc_id and its tf-idf value.
    """
    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    
    doc_scores = bm25.get_scores(terms)

    idx = (-doc_scores).argsort()
    result_docs = tweet_df_doc.iloc[idx].index.values
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search(query, index, model='bm25')
    #print ('\n'.join(result_docs), '\n')
    return result_docs

In [ ]:
print("Queries:")
#query = input()
for query in queries:
  ranked_docs = search(query, index, model='bm25')
  top = 20
  print("\n======================\nQuery: {}".format(query))
  print("\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
  for d_id in ranked_docs[:top]:
      print("{}: {}".format(d_id, tweet_df_doc.loc[d_id]['Tweet']))

Queries:

Query: power outage in carolina

Top 20 results out of 4000 for the searched query:

doc_2607: HOW TO REPORT AN OUTAGE WHERE YOU ARE: 
- North Carolina: 
- South Carolina: 
   
doc_686: Landfall of  is imminent just up the coast from Charleston as power outages begin to mount in coastal South Carolina.

Stay with  for the latest. 
doc_337: Up to 40,000+ power outages in NC now. 

SC has 115,000. 

  
doc_1302: We are having power outages here. Stay safe everyone and check in on family and friends  
doc_1622: With  🌀 passing through today, remember to report any power outages ⚡ directly to your service provider.
➡️ : 
➡️ : 
➡️  
doc_3042: Just in- Horry County EOC Mgr says 7,000+ power outages in the county.  
doc_1850: The current electrical customers in Florida without power is 1.8M, after a peak of 2.6M.

Track power outages in real-time: .

    
doc_2171: I don’t care what anyone says  has the best service.  had least amount of power outages and most people have their powe

# Part 2

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gensim.models import Word2Vec

First we extarct all the words of our dataframe. Then we create word2vec using the library. It converts all the words to a vector (by default of 100 dimensions, 1 per each word)

In [ ]:
all_words = tweet_df.processed_text.values

word2vec = Word2Vec(all_words, min_count=1)

We create N, which is the lenght of the vector. We use any word, in this case the word "be" because we are sure that it will appear in the data. We will use N later to fill an array.

In [ ]:
N = len(word2vec.wv['be'])

We create the fucntion that builds the vector for the document. The vector is is the average number of the words that the document contains.


In [ ]:
def build_vec(text):
    """
    Creates the vector for the documents which is the average of the words that it contains
    
    Argument:
    text -- prosseced text
    
    Returns:
    Vector normalized
    """

    doc_vec = np.zeros(N)
    if len(text)==0:
      return doc_vec
    
    for term in text:
      doc_vec += word2vec.wv[term]
    doc_vec = doc_vec/len(text)
    return doc_vec/la.norm(doc_vec) # we can normalize because we only care about vector direction when computing cosine similarity

We create a new dataframe with the text processed so that we can then add the embeddings.

In [ ]:
word2vec_df = tweet_df_doc[["processed_text"]]

In [ ]:
word2vec_df["embedded"] = word2vec_df.apply(lambda x: build_vec(x.processed_text), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
def rank_documents_word2vec(terms, word2vec_df, sorted=True):
    """
    Perform the ranking of the results of a search based on the word2vec weights
    
    Argument:
    terms -- list of query terms
    word2vec -- df containing the embeddings for word2vec
    sorted -- if we want that the function returns list ordered or not. Yes, by default.
    
    Returns:
    Print the list of ranked documents, the resulting documents and its word2vec value.
    """
    query_vec = build_vec(terms)



    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    
    doc_scores = [[np.dot(word2vec_df.embedded[doc], query_vec), doc] for doc in word2vec_df.index] # only dot product because the 2 vectors are normalized
    if sorted:
      doc_scores.sort(reverse=True)
      result_docs = [x[1] for x in doc_scores]
      #print document titles instead if document id's
      #result_docs=[ title_index[x] for x in result_docs ]
      #print ('\n'.join(result_docs), '\n')
    else:
      result_docs = []
    return result_docs, doc_scores

We print the results for the 5 queries and the word2vec model

In [ ]:
print("Queries:")
#query = input()
for query in queries:
  ranked_docs,_ = rank_documents_word2vec(query, word2vec_df)
  top = 20
  print("\n======================\nQuery: {}".format(query))
  print("\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
  for d_id in ranked_docs[:top]:
      
      print("{}: {}".format(d_id, tweet_df_doc.loc[d_id]['Tweet']))

Queries:

Query: power outage in carolina

Top 20 results out of 4000 for the searched query:

doc_3962: Much of Eastern North Carolina.
The tornado threat is increasing late this morning and into the afternoon across eastern North Carolina. A tornado watch may be needed for coastal and neighboring counties across eastern North Carolina by mid-day.
   
doc_1598:  Watch 548 
Eastern North Carolina, NE South Carolina, SE Virginia, Coastal Waters
Until 1000 PM EDT

* Primary threats include...
A few tornadoes possible
Isolated damaging wind gusts to 70 mph possible
    
 
doc_3299: : 1,700 Flights Canceled Friday, As Storm Heads For Carolinas 
doc_2992: Praying for everyone in the Carolina’s now! Florida…how we holding up? Daytona Beach still has no power 😔

 
 

💪🏾
doc_1140: Legoman is reporting live for the  from Georgia/South Carolina where the effects of  are being felt across the two-state.   
doc_3173: 🧵 All eyes are on . The Category 1 storm is headed directly toward North Carolina